# Identity Resolution Workflow (Blocking, Rule Matching, Evaluation)

This notebook sets up the environment (PyDI, logging, paths) for running blocking and rule-based matching experiments on LR/LS train/val/test splits generated earlier.


# 0. Setup

### 0.1 Logging setup (DEBUG by default during experimentation)

In [70]:
import logging, os
os.makedirs('logs', exist_ok=True)

logging.basicConfig(
    level=logging.DEBUG,
    format='[%(levelname)-5s] %(name)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/workflow.log'),
        logging.StreamHandler()
    ],
    force=True
)
logging.getLogger().debug('Debug logging enabled for workflow notebook')


[DEBUG] root - Debug logging enabled for workflow notebook


### 0.2 Paths: project base, splits (train/val/test), and outputs

In [71]:
from pathlib import Path

# Absolute project base (this notebook lives in the same project directory)
BASE_DIR = Path('/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets')

# Ground-truth splits produced earlier
SPLITS_DIR = BASE_DIR / 'data' / 'output' / 'gt' / 'splits'

# Experiment outputs for this workflow
OUTPUT_DIR = BASE_DIR / 'data' / 'output' / 'workflow'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Convenience: resolved files per edge
LR_TRAIN = SPLITS_DIR / 'gt_LR_train.csv'
LR_VAL   = SPLITS_DIR / 'gt_LR_val.csv'
LR_TEST  = SPLITS_DIR / 'gt_LR_test.csv'

LS_TRAIN = SPLITS_DIR / 'gt_LS_train.csv'
LS_VAL   = SPLITS_DIR / 'gt_LS_val.csv'
LS_TEST  = SPLITS_DIR / 'gt_LS_test.csv'

print('BASE_DIR :', BASE_DIR)
print('SPLITS   :', SPLITS_DIR)
print('OUTPUT   :', OUTPUT_DIR)
print('LR files :', LR_TRAIN.exists(), LR_VAL.exists(), LR_TEST.exists())
print('LS files :', LS_TRAIN.exists(), LS_VAL.exists(), LS_TEST.exists())


BASE_DIR : /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets
SPLITS   : /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/gt/splits
OUTPUT   : /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow
LR files : True True True
LS files : True True True


### 0.3 Import PyDI

In [72]:
import sys, subprocess, importlib
import PyDI  # noqa: F401

print('PyDI ready')

PyDI ready


# 1. Load LR/LS train/val/test splits & exploration


In [73]:
import pandas as pd

# Load six splits
lr_train_df = pd.read_csv(LR_TRAIN)
lr_val_df   = pd.read_csv(LR_VAL)
lr_test_df  = pd.read_csv(LR_TEST)

ls_train_df = pd.read_csv(LS_TRAIN)
ls_val_df   = pd.read_csv(LS_VAL)
ls_test_df  = pd.read_csv(LS_TEST)

splits = {
    'LR_train': lr_train_df,
    'LR_val'  : lr_val_df,
    'LR_test' : lr_test_df,
    'LS_train': ls_train_df,
    'LS_val'  : ls_val_df,
    'LS_test' : ls_test_df,
}

print("Loaded splits (rows, cols):")
for name, df in splits.items():
    lab = df['label'].astype('string').str.upper().str.strip().fillna('') if 'label' in df.columns else None
    label_counts = lab.value_counts().to_dict() if lab is not None else {}
    print(f"  {name:8s}: {df.shape} | labels: {label_counts}")

print("\nSimilarity score summary (min/median/max) where available:")
for name, df in splits.items():
    if 'similarity_score' in df.columns:
        sims = pd.to_numeric(df['similarity_score'], errors='coerce')
        print(f"  {name:8s}: min={sims.min():.3f}, median={sims.median():.3f}, max={sims.max():.3f}")
    else:
        print(f"  {name:8s}: similarity_score column missing")


Loaded splits (rows, cols):
  LR_train: (300, 11) | labels: {'TRUE': 234, 'FALSE': 66}
  LR_val  : (101, 11) | labels: {'TRUE': 78, 'FALSE': 23}
  LR_test : (99, 11) | labels: {'TRUE': 78, 'FALSE': 21}
  LS_train: (300, 11) | labels: {'TRUE': 168, 'FALSE': 132}
  LS_val  : (100, 11) | labels: {'TRUE': 58, 'FALSE': 42}
  LS_test : (100, 11) | labels: {'TRUE': 56, 'FALSE': 44}

Similarity score summary (min/median/max) where available:
  LR_train: min=0.200, median=0.596, max=1.000
  LR_val  : min=0.200, median=0.587, max=1.000
  LR_test : min=0.200, median=0.650, max=1.000
  LS_train: min=0.200, median=0.707, max=1.000
  LS_val  : min=0.200, median=0.741, max=1.000
  LS_test : min=0.200, median=0.725, max=1.000


### 1.1 Missing value check

In [74]:
# Simple NA summary: count and percentage per column
import pandas as pd

def na_summary(df: pd.DataFrame, name: str, max_cols: int = 10):
    na_cnt = df.isna().sum()
    na_cnt = na_cnt[na_cnt > 0].sort_values(ascending=False)
    if len(na_cnt) == 0:
        print(f"{name}: no missing values")
        return
    na_pct = (na_cnt / len(df) * 100).round(2)
    out = pd.DataFrame({'na_count': na_cnt, 'na_%': na_pct})
    print(f"{name}: columns with missing values (top {max_cols})")
    display(out.head(max_cols))

for name, df in splits.items():
    na_summary(df, name)

LR_train: no missing values
LR_val: no missing values
LR_test: no missing values
LS_train: no missing values
LS_val: no missing values
LS_test: no missing values


# 2. Feature preparation


### 2.1 Load source tables for blocking


In [75]:
from PyDI.io import load_xml

CLEAN_DIR = BASE_DIR / 'data' / 'output' / 'clean'

LAHMAN_PATH = (CLEAN_DIR / 'Lahman_Mapped_dedup.xml'
               if (CLEAN_DIR / 'Lahman_Mapped_dedup.xml').exists()
               else BASE_DIR / 'Lahman_Mapped.xml')
REFERENCE_PATH = (CLEAN_DIR / 'Reference_Mapped_dedup.xml'
                  if (CLEAN_DIR / 'Reference_Mapped_dedup.xml').exists()
                  else BASE_DIR / 'Reference_Mapped.xml')
SAVANT_PATH = (CLEAN_DIR / 'Savant_Mapped_dedup.xml'
               if (CLEAN_DIR / 'Savant_Mapped_dedup.xml').exists()
               else BASE_DIR / 'Savant_Mapped.xml')

def _name_initial_key(text: str, max_tokens: int = 3, chars_per_token: int = 2) -> str:
    if not isinstance(text, str):
        return ''
    tokens = text.lower().strip().split()
    pieces = [tok[:chars_per_token] for tok in tokens[:max_tokens] if tok]
    return ''.join(pieces)


def _prepare_table(path, tag):
    df = load_xml(path).convert_dtypes().reset_index(drop=True)

    if 'season_year' in df.columns:
        df['season_year'] = pd.to_numeric(df['season_year'], errors='coerce')
    if 'birth_year' in df.columns:
        df['birth_year'] = pd.to_numeric(df['birth_year'], errors='coerce')

    if 'full_name' in df.columns:
        df['name_prefix_key'] = df['full_name'].astype('string').map(_name_initial_key)
    else:
        df['name_prefix_key'] = ''

    if {'player_id', 'season_year'} <= set(df.columns):
        pid = df['player_id'].astype('string').fillna('NA')
        season = df['season_year'].astype('Int64').astype('string').fillna('NA')
        df['_rid'] = pid + '|' + season + f'|{tag}'
    else:
        df['_rid'] = df.index.map(lambda i: f"{tag}{i:06d}")
    return df

L_full = _prepare_table(LAHMAN_PATH, 'L')
R_full = _prepare_table(REFERENCE_PATH, 'R')
S_full = _prepare_table(SAVANT_PATH, 'S')

print('Loaded tables for blocking:')
for name, df in [('L_full', L_full), ('R_full', R_full), ('S_full', S_full)]:
    cols = [c for c in ['full_name', 'name_prefix_key', 'season_year', 'birth_year', 'team'] if c in df.columns]
    print(f"  {name:6s}: shape={df.shape}, sample cols={cols}")


Loaded tables for blocking:
  L_full: shape=(106553, 25), sample cols=['full_name', 'name_prefix_key', 'season_year', 'birth_year', 'team']
  R_full: shape=(15215, 27), sample cols=['full_name', 'name_prefix_key', 'season_year', 'birth_year', 'team']
  S_full: shape=(6743, 18), sample cols=['full_name', 'name_prefix_key', 'season_year', 'birth_year']


# 2.1 Name Normalization (Reusable function for all blocking methods)

In [76]:
# 2.1 Name Normalization (Reusable function for all blocking methods)
# This function normalizes names to handle encoding issues, backslash escapes, and punctuation
# It will be used by all blocking methods (TokenBlocker, EmbeddingBlocker, etc.)

import re
import unicodedata

def normalize_name_for_blocking(text: str) -> str:
    r"""Normalize name for consistent blocking across all methods
    
    This function handles:
    1. UTF-8 hex escape sequences (e.g., \xc3\xa1 -> á -> a)
    2. Unicode normalization (removes accents: á -> a)
    3. Backslash escape sequences (e.g., o\ brien -> o brien)
    4. Punctuation standardization
    5. Common suffix removal (Jr, Sr, II, III, IV, V)
    
    This normalized name will be used by:
    - Enhanced TokenBlocker (for tokenization)
    - Enhanced EmbeddingBlocker (for embeddings)
    - Enhanced StandardBlocker (for blocking keys)
    - Enhanced SortedNeighbourhoodBlocker (for blocking keys)
    - Any other blocking method that needs normalized names
    """
    if not isinstance(text, str):
        return ''
    
    # Step 1: Decode literal backslash-x-hex patterns (not Python escape sequences)
    # The data contains literal backslash+x+hex (e.g., "\xc3" as 4 characters: \, x, c, 3)
    # We need to match and decode these manually using regex
    def decode_literal_hex_sequence(match):
        r"""Decode literal backslash-x-hex patterns as UTF-8 bytes"""
        hex_bytes = []
        # Extract all hex values from consecutive backslash-x-hex patterns
        for i in range(1, len(match.groups()) + 1):
            hex_str = match.group(i)
            try:
                hex_bytes.append(int(hex_str, 16))
            except ValueError:
                return match.group(0)  # Return original if parsing fails
        
        # Try to decode as UTF-8
        try:
            decoded = bytes(hex_bytes).decode('utf-8')
            return decoded
        except (UnicodeDecodeError, ValueError):
            # If UTF-8 decoding fails, return original
            return match.group(0)
    
    # Match literal backslash+x+hex patterns (e.g., \xHH\xHH for 2-byte UTF-8)
    text = re.sub(r'\\x([0-9a-fA-F]{2})\\x([0-9a-fA-F]{2})', decode_literal_hex_sequence, text)
    # Also handle 3-byte UTF-8 sequences
    text = re.sub(r'\\x([0-9a-fA-F]{2})\\x([0-9a-fA-F]{2})\\x([0-9a-fA-F]{2})', decode_literal_hex_sequence, text)
    
    # Handle any remaining single backslash-x-hex patterns
    def decode_single_hex(match):
        hex_str = match.group(1)
        try:
            return chr(int(hex_str, 16))
        except (ValueError, OverflowError):
            return match.group(0)
    text = re.sub(r'\\x([0-9a-fA-F]{2})', decode_single_hex, text)
    
    # Step 2: Normalize Unicode (NFD: decompose characters, then remove combining marks)
    # This converts "á" to "a" + combining accent, then removes the accent
    text = unicodedata.normalize('NFD', text)
    # Remove combining diacritical marks (accents)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    
    # Step 3: Lowercase and strip
    text = text.lower().strip()
    
    # Step 4: Handle any remaining backslash escape sequences (for cases like "o\ brien")
    text = text.replace('\\ ', ' ')  # Replace backslash+space with space
    text = text.replace('\\', ' ')   # Replace any remaining backslash with space
    
    # Step 5: Standardize punctuation
    text = text.replace('.', '').replace(',', '').replace('-', ' ')
    # Remove apostrophes
    text = text.replace("'", '')
    
    # Step 6: Normalize multiple spaces to single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Step 7: Remove common suffixes (Jr, Sr, II, III, IV, V)
    # This ensures "John Smith Jr" and "John Smith" generate the same blocking keys
    for suffix in [' jr', ' sr', ' ii', ' iii', ' iv', ' v']:
        text = text.replace(suffix, '')
    # Remove any trailing spaces after suffix removal
    text = text.strip()
    
    return text

# Apply normalization to all data tables
# This creates full_name_normalized column that can be reused by all blocking methods
print("\n=== Applying name normalization to all data tables ===")
for name, df in [('L_full', L_full), ('R_full', R_full), ('S_full', S_full)]:
    if 'full_name' in df.columns:
        df['full_name_normalized'] = df['full_name'].astype('string').map(normalize_name_for_blocking)
        print(f"  {name}: Created 'full_name_normalized' column ({len(df):,} records)")
        
        # Show sample normalization for debugging (only for first table)
        if name == 'L_full':
            sample_with_backslash = df[df['full_name'].astype(str).str.contains(r'[\\]', regex=True, na=False)]
            if len(sample_with_backslash) > 0:
                print(f"\n  Sample normalization (first 3 with backslash):")
                for idx, row in sample_with_backslash.head(3).iterrows():
                    orig = row['full_name']
                    norm = row['full_name_normalized']
                    print(f"    '{orig}' -> '{norm}'")
    else:
        print(f"  {name}: No 'full_name' column found, skipping normalization")

print("\n✓ Name normalization complete. All blocking methods can now use 'full_name_normalized' column.")


=== Applying name normalization to all data tables ===
  L_full: Created 'full_name_normalized' column (106,553 records)
  R_full: Created 'full_name_normalized' column (15,215 records)
  S_full: Created 'full_name_normalized' column (6,743 records)

✓ Name normalization complete. All blocking methods can now use 'full_name_normalized' column.


### 2.2.1 Common Helper Functions for Blocking Keys

These functions work on already-normalized names (from `full_name_normalized` column created in section 2.2).


In [77]:
# Common Helper Functions for Blocking Keys
# These functions work on already-normalized names (from full_name_normalized column)

def _name_initial_key_enhanced(text: str, max_tokens: int = 3, chars_per_token: int = 2) -> str:
    """Enhanced name prefix key from normalized name
    
    Note: Input text is already normalized by normalize_name_for_blocking.
    We only need to extract the prefix from the normalized name.
    """
    if not isinstance(text, str):
        return ''
    
    text = text.strip()
    tokens = text.split()
    # Take first max_tokens, first chars_per_token chars each
    pieces = [tok[:chars_per_token] for tok in tokens[:max_tokens] if len(tok) >= chars_per_token]
    return ''.join(pieces) if pieces else ''

def _last_name_initial(text: str) -> str:
    """Extract last name initial from normalized name (handles name order variations)
    
    Note: Input text is already normalized by normalize_name_for_blocking.
    """
    if not isinstance(text, str):
        return ''
    text = text.strip()
    tokens = text.split()
    if tokens:
        return tokens[-1][:2] if len(tokens[-1]) >= 2 else ''
    return ''

print("✓ Common helper functions loaded.")


✓ Common helper functions loaded.


### 2.2.2 Common Evaluation Function for Blocking

Generic function to evaluate blocking strategies on validation set.


In [78]:
# Common Evaluation Function for Blocking
from PyDI.entitymatching.evaluation import EntityMatchingEvaluator

def evaluate_blocking_strategy(candidates_dict, blockers_dict, edges_dict, strategy_name, output_dir):
    """Generic function to evaluate blocking strategies on validation set
    
    Args:
        candidates_dict: Dictionary mapping edge_name to candidate pairs DataFrame
        blockers_dict: Dictionary mapping edge_name to blocker object
        edges_dict: Dictionary mapping edge_name to (left_df, right_df, tags, splits_dict)
        strategy_name: String name of the blocking strategy (for logging)
        output_dir: Path to output directory for evaluation results
    
    Returns:
        Dictionary mapping edge_name to metrics dictionary
    """
    metrics_dict = {}
    
    for edge_name, (_, _, _, splits_dict) in edges_dict.items():
        print(f"\n=== {edge_name}: validation evaluation ({strategy_name}) ===")
        cand_df = candidates_dict[edge_name]
        blocker = blockers_dict[edge_name]
        
        val_df = splits_dict['val'][['id1', 'id2', 'label']].copy()
        metrics = EntityMatchingEvaluator.evaluate_blocking(
            candidate_pairs=cand_df,
            blocker=blocker,
            test_pairs=val_df,
            out_dir=output_dir
        )
        metrics_dict[edge_name] = metrics
        for k, v in metrics.items():
            print(f"  {k}: {v}")
    
    return metrics_dict

print("✓ Common evaluation function loaded.")


✓ Common evaluation function loaded.


### 2.2 NoBlocker baseline (runtime sanity check)


In [79]:
edges = {
    'LR': (L_full, R_full, ('L', 'R'), {'train': lr_train_df, 'val': lr_val_df, 'test': lr_test_df}),
    'LS': (L_full, S_full, ('L', 'S'), {'train': ls_train_df, 'val': ls_val_df, 'test': ls_test_df}),
}


In [80]:
from PyDI.entitymatching.blocking import NoBlocker
import time

# Practical caps to avoid exploding cross products while still gauging cost
MAX_LEFT = 2000
MAX_RIGHT = 2000

noblocker_runtime = {}

for edge_name, (left_df, right_df, (left_tag, right_tag), _) in edges.items():
    full_pairs = len(left_df) * len(right_df)
    print(f"\n=== {edge_name}: NoBlocker baseline ===")
    print(f"Full cartesian product: {full_pairs:,} pairs")

    # Down-sample to keep the experiment tractable
    left_sample = left_df.head(MAX_LEFT)
    right_sample = right_df.head(MAX_RIGHT)
    sampled_pairs = len(left_sample) * len(right_sample)
    truncated = sampled_pairs < full_pairs
    if truncated:
        print(f"Sampling head({MAX_LEFT}) x head({MAX_RIGHT}) -> {sampled_pairs:,} pairs (for runtime check only)")

    start = time.time()
    nb = NoBlocker(left_sample, right_sample, id_column='_rid')
    cand_df = nb.materialize()
    duration = time.time() - start

    noblocker_runtime[edge_name] = {
        'sampled_pairs': sampled_pairs,
        'duration_seconds': duration,
        'truncated': truncated,
    }

    print(f"Materialized {len(cand_df):,} pairs in {duration:.2f}s")
    if truncated:
        print("(Full product is not materialized to avoid OOM; use with caution if needed.)")



=== LR: NoBlocker baseline ===
Full cartesian product: 1,621,203,895 pairs
Sampling head(2000) x head(2000) -> 4,000,000 pairs (for runtime check only)
Materialized 4,000,000 pairs in 0.20s
(Full product is not materialized to avoid OOM; use with caution if needed.)

=== LS: NoBlocker baseline ===
Full cartesian product: 718,486,879 pairs
Sampling head(2000) x head(2000) -> 4,000,000 pairs (for runtime check only)
Materialized 4,000,000 pairs in 0.10s
(Full product is not materialized to avoid OOM; use with caution if needed.)




- LR full cartesian product: 1,621,203,895 pairs  
  - Sampled head(2000) × head(2000) → 4,000,000 pairs  
  - Materialized in 0.14 s (full product skipped to avoid OOM)

- LS full cartesian product: 718,486,879 pairs  
  - Sampled head(2000) × head(2000) → 4,000,000 pairs  
  - Materialized in 0.11 s (full product skipped to avoid OOM)

**Takeaway:** NoBlocker confirms the raw search space is >10⁹ pairs, so exhaustive matching is infeasible; blocking is mandatory before scoring/matching.

### 2.3a StandardBlocker — full_name initials


In [81]:
standard_blockers_name_prefix = {}
candidates_name_prefix_df = {}


In [82]:
from PyDI.entitymatching.blocking import StandardBlocker

standard_blockers_name_prefix = {}
candidates_name_prefix_df = {}

for edge_name, (left_df, right_df, (left_tag, right_tag), _) in edges.items():
    print(f"\n=== {edge_name}: StandardBlocker (on full_name initials) — materialize ===")
    blocker = StandardBlocker(
        left_df, right_df,
        on=['name_prefix_key'],
        output_dir=OUTPUT_DIR / 'blocking-evaluation',
        id_column='_rid'
    )
    standard_blockers_name_prefix[edge_name] = blocker

    # Materialize all candidate pairs (exercise-style)
    cand_df = blocker.materialize()
    candidates_name_prefix_df[edge_name] = cand_df

    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(cand_df) / total_pairs) if total_pairs else float('nan')

    print(f"  Generated: {len(cand_df):,} candidates | reduction ratio={rr:.6f}")
    try:
        display(cand_df.head(10))
    except Exception:
        print(cand_df.head(10).to_string(index=False))



=== LR: StandardBlocker (on full_name initials) — materialize ===


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset1: 106553 records
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset2: 15215 records
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 8575 blocking keys for first dataset
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 2895 blocking keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Joining blocking key values: 8575 x 2895 blocks
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 2723 blocks from blocking keys
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Block size distribution:
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 147         4
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 132         1

  Generated: 449,131 candidates | reduction ratio=0.999723


,id1,id2,block_key
0,1100041875|1875|L,5479892014|2014|R,joda
1,1100041875|1875|L,5479892015|2015|R,joda
2,1100041875|1875|L,5479892016|2016|R,joda
3,1100041875|1875|L,5479892017|2017|R,joda
4,1100041875|1875|L,5479892018|2018|R,joda
5,1100041875|1875|L,5479892019|2019|R,joda
6,1100041875|1875|L,5479892020|2020|R,joda
7,1100041875|1875|L,5479892021|2021|R,joda
8,1100041875|1875|L,5479892022|2022|R,joda
9,1100041875|1875|L,5479892023|2023|R,joda


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset1: 106553 records



=== LS: StandardBlocker (on full_name initials) — materialize ===


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset2: 6743 records
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 8575 blocking keys for first dataset
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 1585 blocking keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Joining blocking key values: 8575 x 1585 blocks
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 1533 blocks from blocking keys
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Block size distribution:
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 69          1
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 67          4
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 57          9
[DEBUG] PyDI.entitymatching.blocking.standar

  Generated: 193,269 candidates | reduction ratio=0.999731


,id1,id2,block_key
0,1100041875|1875|L,5479892015|2015|S,pama
1,1100041875|1875|L,5479892016|2016|S,pama
2,1100041875|1875|L,5479892017|2017|S,pama
3,1100041875|1875|L,5479892018|2018|S,pama
4,1100041875|1875|L,5479892019|2019|S,pama
5,1100041875|1875|L,5479892020|2020|S,pama
6,1100041875|1875|L,5479892021|2021|S,pama
7,1100041875|1875|L,5479892022|2022|S,pama
8,1100041875|1875|L,5479892023|2023|S,pama
9,1100041875|1875|L,5479892024|2024|S,pama


### 2.3b Blocking evaluation (validation GT)



In [83]:
# 2.4a Blocking evaluation (validation GT)
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_name_prefix = evaluate_blocking_strategy(
    candidates_dict=candidates_name_prefix_df,
    blockers_dict=standard_blockers_name_prefix,
    edges_dict=edges,
    strategy_name="StandardBlocker - Full Name Initials",
    output_dir=OUTPUT_DIR / 'blocking-evaluation'
)



=== LR: validation evaluation (StandardBlocker - Full Name Initials) ===


[INFO ] root -   Pair Completeness: 0.705
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.999723
[INFO ] root -   True Matches Found: 55/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 0.7051282051282052
  pair_quality: 0.0001224587035853682
  reduction_ratio: 0.9997229645195246
  total_candidates: 449131
  total_possible_pairs: 1621203895
  true_positives_found: 55
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T17:51:33.648285
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (StandardBlocker - Full Name Initials) ===


[INFO ] root -   Pair Completeness: 0.466
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.999731
[INFO ] root -   True Matches Found: 27/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 0.46551724137931033
  pair_quality: 0.00013970165934526488
  reduction_ratio: 0.9997310055261287
  total_candidates: 193269
  total_possible_pairs: 718486879
  true_positives_found: 27
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T17:51:42.031708
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


#### 2.3c Blocking Evaluation Summary

- Candidate volume dropped by orders of magnitude (LR: 20.6M → 0.45M; LS: 9.5M → 0.19M), confirming that the name-initial key is an extremely aggressive filter.
- Pair Completeness fell to 70.5% (LR) and 46.6% (LS); roughly one-third to one-half of validation matches are being lost because the initials do not align (abbreviations, suffixes, token order issues, etc.).
- Pair Quality is reported as ≈0 because true matches are sparse relative to the remaining candidate pool (≈10⁻⁴); this is expected for a high-reduction blocker and not a correctness issue.
- Action items:
  - Add a complementary, more recall-focused blocker (e.g., name initials OR season-year + first-letter match, or initials plus coarse birth-year buckets).
  - Audit missed matches to learn which name variations are not captured (jr/sr, two-letter nicknames, middle initials, spelling variants) and extend the key accordingly.
  - Keep the initial-based blocker in the final pipeline as a low-cost channel, but union it with a higher-recall channel before scoring/matching.

### 2.3d Problem Analysis and Solutions

#### **Identified Issues**

1. **Low Recall**: StandardBlocker achieves only 70.5% (LR) and 46.6% (LS) Pair Completeness, losing significant true matches.
2. **Name Variation Challenges**: Suffixes (Jr/Sr/II/III), punctuation differences, and token order variations cause mismatched blocking keys.
3. **Trade-off**: Strict blockers have high reduction but low recall; high-recall blockers (TokenBlocker: 97.4% LR, 100% LS) generate millions of candidates.

#### **Proposed Solutions (StandardBlocker-based)**

1. **Multi-Strategy Union**: Combine multiple StandardBlocker strategies with OR logic:
   - Enhanced name prefix key (normalized, handles suffixes and punctuation)
   - Loose name prefix (first 2 chars of first 2 words for broader coverage)
   - Last name initial (handles name order variations)
   - Name prefix + season_year (more specific matching)
   - Union all candidate sets and deduplicate to maximize recall.

2. **Name Normalization Enhancement**:
   - Remove common suffixes (Jr, Sr, II, III, IV, V) before generating blocking keys
   - Normalize punctuation (remove periods, commas, hyphens) to handle formatting differences
   - Extract last name initial as a fallback for handling name order variations

3. **Relaxed Blocking Keys**:
   - Use `name_prefix_loose` (2 tokens instead of 3) for broader coverage
   - Create separate blocking keys for different name components (first name, last name)
   - Combine with season_year for more specific matching when available

### 2.4 Enhanced StandardBlocker — Multi-Strategy Union

- Implements multiple StandardBlocker strategies with union to maximize recall:
  - Enhanced name prefix key (normalized, handles suffixes and punctuation)
  - Loose name prefix (2 tokens for broader coverage)
  - Last name initial (handles name order variations)
  - Name prefix + season_year (more specific matching)
- Combines all candidate sets with OR logic to capture matches missed by individual strategies.
- Evaluation separated in next cell to track recall improvement.


#### Helper Keys for Enhanced Blocking

- Adds coarse birth-year buckets (`birth_decade`) to tolerate ±1 year noise (available for future use).
- Builds a relaxed name-prefix key (`name_prefix_loose`) using up to two tokens for broader coverage.
- Computed once on `L_full`, `R_full`, `S_full`; reused by enhanced StandardBlocker strategies.


In [84]:
import pandas as pd


def _birth_decade(series: pd.Series) -> pd.Series:
    values = pd.to_numeric(series, errors='coerce')
    return (values // 10 * 10).astype('Int64')


# Loose name prefix key function (works on already-normalized names)
# Since full_name_normalized is already normalized, we only need to extract prefix
def _name_initial_key_loose(text: str, max_tokens: int = 2, chars_per_token: int = 2) -> str:
    """Loose name prefix key from normalized name (2 tokens for broader coverage)
    
    Note: Input text is already normalized by normalize_name_for_blocking.
    """
    if not isinstance(text, str):
        return ''
    tokens = text.strip().split()  # Already lowercase from normalization
    pieces = [tok[:chars_per_token] for tok in tokens[:max_tokens] if tok]
    return ''.join(pieces)


# Create helper columns using normalized names (from 2.1)
# Note: This cell should be run AFTER section 2.2 (Name Normalization)
for df in [L_full, R_full, S_full]:
    if 'birth_year' in df.columns:
        df['birth_decade'] = _birth_decade(df['birth_year'])
    else:
        df['birth_decade'] = pd.NA
    
    # Use full_name_normalized column (created in 2.1) for consistent normalization
    if 'full_name_normalized' in df.columns:
        df['name_prefix_loose'] = df['full_name_normalized'].astype('string').map(_name_initial_key_loose)
    else:
        print(f"Warning: full_name_normalized column not found. Please run section 2.2 (Name Normalization) first.")
        # Fallback to full_name if normalized column doesn't exist (for backward compatibility)
        df['name_prefix_loose'] = df['full_name'].astype('string').map(_name_initial_key_loose)

print('Helper columns added: birth_decade, name_prefix_loose (using full_name_normalized)')


Helper columns added: birth_decade, name_prefix_loose (using full_name_normalized)


In [85]:
# Materialize Enhanced StandardBlocker (selective multi-strategy union)
# LR: Uses strategies 1, 2, 3, and 4 (Strategy 3 enabled to recover missed matches)
# LS: Uses strategies 1, 2, and 4 (Strategy 3 skipped since already 100% recall)
# Uses full_name_normalized column (created in 2.2) for consistent normalization
# Note: Helper functions (_name_initial_key_enhanced, _last_name_initial, _name_initial_key_loose)
#       are defined in section 2.2.1 (_name_initial_key_enhanced, _last_name_initial) and section 2.5a (_name_initial_key_loose)
from PyDI.entitymatching.blocking import StandardBlocker
import pandas as pd

# Enhanced StandardBlocker with selective multi-strategy union
# Strategy 3 (last name initial) is conditionally enabled:
# - Enabled for LR edge to recover missed matches (target: >97% recall)
# - Skipped for LS edge since it already achieves 100% recall without it
standard_blockers_enhanced = {}
candidates_enhanced = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: Enhanced StandardBlocker (optimized multi-strategy) — materialize ===")
    
    # Prepare enhanced blocking keys for both dataframes
    # Use full_name_normalized column (created in 2.1) for consistent normalization
    for df_label, df_obj in [('left', left_df), ('right', right_df)]:
        # Check if full_name_normalized exists (should be created in 2.1)
        if 'full_name_normalized' not in df_obj.columns:
            print(f"  Warning: full_name_normalized column not found for {df_label} table.")
            print(f"  Please run section 2.2 (Name Normalization) first.")
            continue
        
        # Enhanced name prefix key (from normalized name)
        if 'name_prefix_key_enhanced' not in df_obj.columns:
            df_obj['name_prefix_key_enhanced'] = df_obj['full_name_normalized'].astype('string').map(_name_initial_key_enhanced)
        
        # Last name initial column preparation (from normalized name)
        if 'last_name_initial' not in df_obj.columns:
            df_obj['last_name_initial'] = df_obj['full_name_normalized'].astype('string').map(_last_name_initial)
        
        # Ensure name_prefix_loose exists (from normalized name)
        if 'name_prefix_loose' not in df_obj.columns:
            df_obj['name_prefix_loose'] = df_obj['full_name_normalized'].astype('string').map(_name_initial_key_loose)
    
    # Strategy 1: Enhanced name prefix key (normalized, handles suffixes)
    print(f"  Strategy 1: Enhanced name prefix key (normalized)")
    blocker1 = StandardBlocker(
        left_df, right_df,
        on=['name_prefix_key_enhanced'],
        output_dir=OUTPUT_DIR / 'blocking-evaluation',
        id_column='_rid'
    )
    cand1 = blocker1.materialize()
    print(f"    Generated: {len(cand1):,} candidates")
    
    # Strategy 2: Loose name prefix (broader coverage)
    print(f"  Strategy 2: Loose name prefix (2 tokens)")
    blocker2 = StandardBlocker(
        left_df, right_df,
        on=['name_prefix_loose'],
        output_dir=OUTPUT_DIR / 'blocking-evaluation',
        id_column='_rid'
    )
    cand2 = blocker2.materialize()
    print(f"    Generated: {len(cand2):,} candidates")
    
    # Strategy 3: Last name initial (SELECTIVE - only for LR edge to recover missed matches)
    # For LR: Strategy 3 helps recover matches lost due to name order variations
    # For LS: Skipped since LS already achieves 100% recall without it
    cand3 = None
    if edge_name == 'LR':
        print(f"  Strategy 3: Last name initial (ENABLED for LR to recover missed matches)")
        blocker3 = StandardBlocker(
            left_df, right_df,
            on=['last_name_initial'],
            output_dir=OUTPUT_DIR / 'blocking-evaluation',
            id_column='_rid'
        )
        cand3 = blocker3.materialize()
        print(f"    Generated: {len(cand3):,} candidates")
    else:
        print(f"  Strategy 3: Last name initial (SKIPPED for {edge_name} - already 100% recall)")
    
    # Strategy 4: Enhanced name prefix + season_year (more specific)
    print(f"  Strategy 4: Enhanced name prefix + season_year")
    blocker4 = StandardBlocker(
        left_df, right_df,
        on=['name_prefix_key_enhanced', 'season_year'],
        output_dir=OUTPUT_DIR / 'blocking-evaluation',
        id_column='_rid'
    )
    cand4 = blocker4.materialize()
    print(f"    Generated: {len(cand4):,} candidates")
    
    # Union strategies based on edge_name
    # LR: Union strategies 1, 2, 3, and 4 (Strategy 3 helps recover missed matches)
    # LS: Union strategies 1, 2, and 4 (Strategy 3 skipped since already 100% recall)
    strategy_list = [cand1, cand2, cand4]
    strategy_names = ['Strategy 1', 'Strategy 2', 'Strategy 4']
    if cand3 is not None:
        strategy_list.append(cand3)
        strategy_names.append('Strategy 3')
    
    all_candidates = pd.concat(strategy_list, ignore_index=True)
    candidates_union = all_candidates.drop_duplicates(subset=['id1', 'id2']).reset_index(drop=True)
    
    # Store the union result
    standard_blockers_enhanced[edge_name] = blocker1  # Use first blocker for evaluation reference
    candidates_enhanced[edge_name] = candidates_union
    
    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(candidates_union) / total_pairs) if total_pairs else float('nan')
    
    print(f"\n  === Combined Results ({', '.join(strategy_names)}) ===")
    print(f"  Total unique candidates: {len(candidates_union):,}")
    print(f"  Reduction ratio: {rr:.6f}")
    strategy_sizes = [len(cand1), len(cand2), len(cand4)]
    if cand3 is not None:
        strategy_sizes.insert(2, len(cand3))  # Insert Strategy 3 size in correct position
        print(f"  Individual strategy sizes: {len(cand1):,} + {len(cand2):,} + {len(cand3):,} + {len(cand4):,}")
    else:
        print(f"  Individual strategy sizes: {len(cand1):,} + {len(cand2):,} + {len(cand4):,} (Strategy 3 skipped)")


=== LR: Enhanced StandardBlocker (optimized multi-strategy) — materialize ===


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset1: 106553 records


  Strategy 1: Enhanced name prefix key (normalized)


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset2: 15215 records
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 8306 blocking keys for first dataset
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 2786 blocking keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Joining blocking key values: 8306 x 2786 blocks
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 2778 blocks from blocking keys
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Block size distribution:
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 149         4
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 133         1
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 102         9
[DEBUG] PyDI.entitymatching.blocking.standa

    Generated: 486,265 candidates
  Strategy 2: Loose name prefix (2 tokens)


...

    Generated: 210,006 candidates
  Strategy 2: Loose name prefix (2 tokens)


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset2: 6743 records
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 8240 blocking keys for first dataset
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 1543 blocking keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Joining blocking key values: 8240 x 1543 blocks
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 1536 blocks from blocking keys
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Block size distribution:
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 66          1
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 59          9
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 59          4
[DEBUG] PyDI.entitymatching.blocking.standar

    Generated: 209,958 candidates
  Strategy 3: Last name initial (SKIPPED for LS - already 100% recall)
  Strategy 4: Enhanced name prefix + season_year


[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Creating blocking key values for dataset2: 6743 records
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 95008 blocking keys for first dataset
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 6178 blocking keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Joining blocking key values: 95008 x 6178 blocks
[INFO ] PyDI.entitymatching.blocking.standard.StandardBlocker - created 6152 blocks from blocking keys
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Block size distribution:
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 4966        1
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 606         2
[DEBUG] PyDI.entitymatching.blocking.standard.StandardBlocker - 303         4
[DEBUG] PyDI.entitymatching.blocking.stand

    Generated: 9,401 candidates

  === Combined Results (Strategy 1, Strategy 2, Strategy 4) ===
  Total unique candidates: 215,708
  Reduction ratio: 0.999700
  Individual strategy sizes: 210,006 + 209,958 + 9,401 (Strategy 3 skipped)


#### 2.4 Validation evaluation (Enhanced StandardBlocker - multi-strategy)

In [86]:
# 2.5b Validation evaluation (Enhanced StandardBlocker - multi-strategy)
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_enhanced = evaluate_blocking_strategy(
    candidates_dict=candidates_enhanced,
    blockers_dict=standard_blockers_enhanced,
    edges_dict=edges,
    strategy_name="Enhanced StandardBlocker - Multi-Strategy",
    output_dir=OUTPUT_DIR / 'blocking-evaluation'
)



=== LR: validation evaluation (Enhanced StandardBlocker - Multi-Strategy) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.988410
[INFO ] root -   True Matches Found: 78/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 4.151109761941306e-06
  reduction_ratio: 0.9884097515075363
  total_candidates: 18790156
  total_possible_pairs: 1621203895
  true_positives_found: 78
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T17:55:31.121655
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (Enhanced StandardBlocker - Multi-Strategy) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.999700
[INFO ] root -   True Matches Found: 58/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 0.0002688820071578245
  reduction_ratio: 0.9996997746148124
  total_candidates: 215708
  total_possible_pairs: 718486879
  true_positives_found: 58
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:00:00.564167
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


### 2.5 Sorted neighbourhood blocking

#### 2.5a Materialize SortedNeighbourhoodBlocker (window=20 on name_prefix_key)

In [87]:
from PyDI.entitymatching.blocking import SortedNeighbourhoodBlocker

sorted_blockers_name = {}
candidates_sorted = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: SortedNeighbourhoodBlocker (name_prefix_key, window=20) ===")

    # Ensure the key column exists on both sides (re-use earlier prefix derivation if needed)
    for label, df_obj in [("left", left_df), ("right", right_df)]:
        if "name_prefix_key" not in df_obj.columns and "full_name" in df_obj.columns:
            df_obj["name_prefix_key"] = (
                df_obj["full_name"]
                .astype("string")
                .str.lower()
                .str.replace(r"\s+", " ", regex=True)
            )
            print(f"  [{edge_name}] backfilled name_prefix_key for {label} table")

    blocker = SortedNeighbourhoodBlocker(
        df_left=left_df,
        df_right=right_df,
        key="name_prefix_key",
        window=20,                         # adjust to trade recall vs. runtime
        id_column="_rid",
        output_dir=str(OUTPUT_DIR / "blocking-evaluation")
    )
    sorted_blockers_name[edge_name] = blocker

    cand_df = blocker.materialize()
    candidates_sorted[edge_name] = cand_df

    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(cand_df) / total_pairs) if total_pairs else float("nan")
    print(f"  Generated: {len(cand_df):,} candidates | reduction ratio={rr:.6f}")

[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset1: 106553 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset2: 15215 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Sorting combined dataset with 121768 records



=== LR: SortedNeighbourhoodBlocker (name_prefix_key, window=20) ===


[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - created sorted neighbourhood with window size 20
[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - created 1 sorted sequence from 121768 records
[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Debug results written to file: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/debugResultsBlocking_SortedNeighbourhoodBlocker.csv
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating candidate record pairs from sorted neighbourhood with window 20
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset1: 106553 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset2: 6743 records
[DEBUG] PyDI.entitymatching.blo

  Generated: 434,493 candidates | reduction ratio=0.999732

=== LS: SortedNeighbourhoodBlocker (name_prefix_key, window=20) ===
  Generated: 219,574 candidates | reduction ratio=0.999694


#### 2.5b Validate SortedNeighbourhoodBlocker with ground-truth splits

In [88]:
# 2.6b Validate SortedNeighbourhoodBlocker with Ground-Truth Splits
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_sorted = evaluate_blocking_strategy(
    candidates_dict=candidates_sorted,
    blockers_dict=sorted_blockers_name,
    edges_dict=edges,
    strategy_name="SortedNeighbourhoodBlocker",
    output_dir=OUTPUT_DIR / "blocking-evaluation"
)


=== LR: validation evaluation (SortedNeighbourhoodBlocker) ===


[INFO ] root -   Pair Completeness: 0.846
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.999732
[INFO ] root -   True Matches Found: 66/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 0.8461538461538461
  pair_quality: 0.00015190118137691516
  reduction_ratio: 0.9997319936120681
  total_candidates: 434493
  total_possible_pairs: 1621203895
  true_positives_found: 66
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T18:00:09.018499
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (SortedNeighbourhoodBlocker) ===


[INFO ] root -   Pair Completeness: 0.948
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.999694
[INFO ] root -   True Matches Found: 55/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 0.9482758620689655
  pair_quality: 0.00025048503010374637
  reduction_ratio: 0.9996943938624104
  total_candidates: 219574
  total_possible_pairs: 718486879
  true_positives_found: 55
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:00:17.189227
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


### SortedNeighbourhoodBlocker Evaluation Summary
- **Recall vs. coverage**
  - `LR`: Pair Completeness 0.846 (66/78 matches recovered).  
  - `LS`: Pair Completeness 0.948 (55/58).  
  - Sliding window picks up most near-duplicate names, but still misses ≈15% of LR matches; a larger window (or a second looser key) may be needed.

- **Candidate volume**
  - `LR`: 434,493 candidates (reduction ratio ≈ 0.99973 → ~0.27% of the full Cartesian).  
  - `LS`: 219,574 candidates (reduction ratio ≈ 0.99969).  
  - Runtime is manageable and far smaller than token blocking (millions of pairs), while still much larger than strict equality keys.

- **Precision proxy**
  - Pair Quality around 1.5e-4 (LR) and 2.5e-4 (LS); true matches remain sparse in the blocks, so downstream scoring is still required.

- **Takeaways**
  - SortedNeighbourhoodBlocker is a solid middle ground: recall far higher than initials-only keys, with candidate sets an order of magnitude smaller than the token blocker.  
  - Tune with `window` size or a secondary sort key (e.g., initials + birth decade) to balance extra recall vs. candidate growth.  
  - Combine with the initials blocker (union) for higher coverage before matching, or use SNB as the high-recall channel when token blocking is too costly.

### 2.6 Enhanced SortedNeighbourhoodBlocker (Improved)

This section implements an improved version of SortedNeighbourhoodBlocker with:
- **Enhanced name normalization**: Uses `name_prefix_key_enhanced` (handles suffixes, punctuation) instead of basic `name_prefix_key`
- **Larger window size**: Increased from 20 to 80 to capture more near-duplicate names


#### 2.6a Materialize Enhanced SortedNeighbourhoodBlocker (enhanced key, window=80)


In [89]:
from PyDI.entitymatching.blocking import SortedNeighbourhoodBlocker

# Note: Helper function _name_initial_key_enhanced is defined in section 2.2.1

sorted_blockers_enhanced_snb = {}
candidates_sorted_enhanced = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: Enhanced SortedNeighbourhoodBlocker (using unified normalization, window=80) ===")
    
    # Prepare enhanced name prefix key for both dataframes
    # Use full_name_normalized column (created in 2.1) for consistent normalization
    for label, df_obj in [("left", left_df), ("right", right_df)]:
        # Check if full_name_normalized exists (should be created in 2.1)
        if "full_name_normalized" not in df_obj.columns:
            print(f"  Warning: full_name_normalized column not found for {label} table.")
            print(f"  Please run section 2.2 (Name Normalization) first.")
            continue
        
        if "name_prefix_key_enhanced" not in df_obj.columns:
            df_obj["name_prefix_key_enhanced"] = df_obj["full_name_normalized"].astype("string").map(_name_initial_key_enhanced)
            print(f"  [{edge_name}] Created name_prefix_key_enhanced for {label} table (from full_name_normalized)")
    
    blocker = SortedNeighbourhoodBlocker(
        df_left=left_df,
        df_right=right_df,
        key="name_prefix_key_enhanced",  # Use enhanced key (now with backslash fix)
        window=80,                         # Increased from 40 to 80 to capture more matches with same key
        id_column="_rid",
        output_dir=str(OUTPUT_DIR / "blocking-evaluation")
    )
    sorted_blockers_enhanced_snb[edge_name] = blocker
    
    cand_df = blocker.materialize()
    candidates_sorted_enhanced[edge_name] = cand_df
    
    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(cand_df) / total_pairs) if total_pairs else float("nan")
    print(f"  Generated: {len(cand_df):,} candidates | reduction ratio={rr:.6f}")



=== LR: Enhanced SortedNeighbourhoodBlocker (using unified normalization, window=80) ===

[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset1: 106553 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset2: 15215 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Sorting combined dataset with 121768 records
[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - created sorted neighbourhood with window size 80
[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - created 1 sorted sequence from 121768 records


[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Debug results written to file: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/debugResultsBlocking_SortedNeighbourhoodBlocker.csv
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating candidate record pairs from sorted neighbourhood with window 80
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset1: 106553 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating sort keys for dataset2: 6743 records
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Sorting combined dataset with 113296 records


  Generated: 1,986,017 candidates | reduction ratio=0.998775

=== LS: Enhanced SortedNeighbourhoodBlocker (using unified normalization, window=80) ===


[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - created sorted neighbourhood with window size 80
[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - created 1 sorted sequence from 113296 records
[INFO ] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Debug results written to file: /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/debugResultsBlocking_SortedNeighbourhoodBlocker.csv
[DEBUG] PyDI.entitymatching.blocking.sorted_neighbourhood.SortedNeighbourhoodBlocker - Creating candidate record pairs from sorted neighbourhood with window 80


  Generated: 964,403 candidates | reduction ratio=0.998658


#### 2.6b Validate Enhanced SortedNeighbourhoodBlocker with ground-truth splits


In [90]:
# 2.7b Validation evaluation (Enhanced SortedNeighbourhoodBlocker)
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_sorted_enhanced = evaluate_blocking_strategy(
    candidates_dict=candidates_sorted_enhanced,
    blockers_dict=sorted_blockers_enhanced_snb,
    edges_dict=edges,
    strategy_name="Enhanced SortedNeighbourhoodBlocker",
    output_dir=OUTPUT_DIR / 'blocking-evaluation'
)



=== LR: validation evaluation (Enhanced SortedNeighbourhoodBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.998775
[INFO ] root -   True Matches Found: 78/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 3.92745882839875e-05
  reduction_ratio: 0.9987749739523047
  total_candidates: 1986017
  total_possible_pairs: 1621203895
  true_positives_found: 78
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T18:00:43.689656
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (Enhanced SortedNeighbourhoodBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.998658
[INFO ] root -   True Matches Found: 58/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 6.0140833240875444e-05
  reduction_ratio: 0.9986577305331695
  total_candidates: 964403
  total_possible_pairs: 718486879
  true_positives_found: 58
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:01:21.297119
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


#### 2.6d Analyze missed matches (Enhanced SortedNeighbourhoodBlocker)

**Note**: This analysis was performed during development when Enhanced SortedNeighbourhoodBlocker had lower recall. The current version achieves 100% recall for both LR and LS edges with window=80. This section is kept for reference to understand the improvements made.s


In [91]:
# 2.6d Analyze missed matches for Enhanced SortedNeighbourhoodBlocker (LR edge only)
# Purpose: Understand why 10 true matches are not captured by the Enhanced SortedNeighbourhoodBlocker

import pandas as pd

print("=== Analyzing Missed Matches for Enhanced SortedNeighbourhoodBlocker (LR Edge) ===\n")

# Get validation set for LR edge
edge_name = 'LR'
left_df, right_df, _, splits_dict = edges[edge_name]
val_df = splits_dict['val'][['id1', 'id2', 'label']].copy()

# Debug: Check label column values
print("Debug: Checking validation set structure...")
print(f"  Total validation pairs: {len(val_df)}")
print(f"  Label column unique values: {val_df['label'].unique()}")
print(f"  Label column value counts:\n{val_df['label'].value_counts()}")

# Normalize label values (handle case variations, whitespace, etc.)
val_df['label_normalized'] = val_df['label'].astype(str).str.strip().str.upper()
val_true = val_df[val_df['label_normalized'] == 'TRUE'].copy()

print(f"\n  After normalization - TRUE matches: {len(val_true)}")

# Check if candidates_sorted_enhanced exists
if 'candidates_sorted_enhanced' not in globals() or edge_name not in candidates_sorted_enhanced:
    print(f"ERROR: candidates_sorted_enhanced['{edge_name}'] not found!")
    print("Please run cell 2.6a (Materialize Enhanced SortedNeighbourhoodBlocker) first.")
else:
    cand_df = candidates_sorted_enhanced[edge_name]
    print(f"\nDebug: Candidate pairs info...")
    print(f"  Total candidate pairs: {len(cand_df)}")
    print(f"  Candidate columns: {list(cand_df.columns)}")
    
    # Find which true matches are missing from candidates
    cand_set = set(zip(cand_df['id1'], cand_df['id2']))
    print(f"  Candidate set size: {len(cand_set)}")
    
    # Check if any true matches are in candidates
    if len(val_true) > 0:
        val_true_in_cand = val_true.apply(
            lambda row: (row['id1'], row['id2']) in cand_set, axis=1
        )
        print(f"  True matches found in candidates: {val_true_in_cand.sum()} / {len(val_true)}")
        
        missed = val_true[~val_true_in_cand].copy()
    else:
        missed = val_true.copy()
    
    print(f"\nTotal true matches in validation set: {len(val_true)}")
    print(f"True matches found in candidates: {len(val_true) - len(missed)}")
    print(f"True matches MISSED: {len(missed)}\n")
    
    if len(missed) > 0:
        # Prepare left dataframe columns
        left_cols = ['_rid', 'full_name', 'season_year', 'birth_year', 'name_prefix_key_enhanced']
        left_df_subset = left_df[left_cols].copy()
        left_rename_dict = {'_rid': 'id1'}
        for col in left_cols:
            if col != '_rid':
                left_rename_dict[col] = f'left_{col}'
        left_df_subset = left_df_subset.rename(columns=left_rename_dict)
        
        # Prepare right dataframe columns
        right_cols = ['_rid', 'full_name', 'season_year', 'birth_year', 'name_prefix_key_enhanced']
        right_df_subset = right_df[right_cols].copy()
        right_rename_dict = {'_rid': 'id2'}
        for col in right_cols:
            if col != '_rid':
                right_rename_dict[col] = f'right_{col}'
        right_df_subset = right_df_subset.rename(columns=right_rename_dict)
        
        # Merge
        missed_analysis = missed.merge(
            left_df_subset,
            on='id1',
            how='left'
        ).merge(
            right_df_subset,
            on='id2',
            how='left'
        )
        
        # Calculate differences and blocking key mismatches
        missed_analysis['key_match'] = (
            missed_analysis['left_name_prefix_key_enhanced'] == 
            missed_analysis['right_name_prefix_key_enhanced']
        )
        missed_analysis['season_year_match'] = (
            missed_analysis['left_season_year'] == missed_analysis['right_season_year']
        )
        missed_analysis['season_year_diff'] = (
            missed_analysis['left_season_year'] - missed_analysis['right_season_year']
        ).abs()
        missed_analysis['birth_year_diff'] = (
            missed_analysis['left_birth_year'] - missed_analysis['right_birth_year']
        ).abs()
        
        # Check if keys are similar (for SortedNeighbourhoodBlocker, even if keys don't match exactly,
        # they might be close in sorted order)
        def key_similarity(key1, key2):
            """Check if two keys are similar (for sorting proximity)"""
            if pd.isna(key1) or pd.isna(key2) or key1 == '' or key2 == '':
                return False
            # Check if one key is a prefix of another
            if key1.startswith(key2) or key2.startswith(key1):
                return True
            # Check if they share common prefix
            min_len = min(len(key1), len(key2))
            if min_len > 0:
                common_prefix = sum(1 for i in range(min_len) if key1[i] == key2[i])
                return common_prefix >= min_len * 0.5  # At least 50% common prefix
            return False
        
        missed_analysis['key_similar'] = missed_analysis.apply(
            lambda row: key_similarity(
                str(row['left_name_prefix_key_enhanced']), 
                str(row['right_name_prefix_key_enhanced'])
            ), axis=1
        )
        
        # Display detailed analysis
        print("=" * 80)
        print("DETAILED ANALYSIS OF MISSED MATCHES")
        print("=" * 80)
        
        for idx, row in missed_analysis.iterrows():
            print(f"\n--- Missed Match #{idx + 1} ---")
            print(f"Left ID:  {row['id1']}")
            print(f"Right ID: {row['id2']}")
            print(f"\nNames:")
            print(f"  Left:  {row['left_full_name']}")
            print(f"  Right: {row['right_full_name']}")
            print(f"\nSorting Keys (name_prefix_key_enhanced):")
            print(f"  Left:  '{row['left_name_prefix_key_enhanced']}'")
            print(f"  Right: '{row['right_name_prefix_key_enhanced']}'")
            print(f"    Exact match: {row['key_match']}")
            print(f"    Similar (for sorting): {row['key_similar']}")
            print(f"\nTemporal Information:")
            print(f"  Season year (left):  {row['left_season_year']}")
            print(f"  Season year (right): {row['right_season_year']}")
            print(f"    Match: {row['season_year_match']} (diff: {row['season_year_diff']})")
            print(f"  Birth year (left):   {row['left_birth_year']}")
            print(f"  Birth year (right):  {row['right_birth_year']}")
            print(f"    Diff: {row['birth_year_diff']}")
        
        # Summary statistics
        print("\n" + "=" * 80)
        print("SUMMARY STATISTICS")
        print("=" * 80)
        print(f"\nSorting Key Analysis:")
        print(f"  Keys match exactly: {sum(missed_analysis['key_match'])} / {len(missed_analysis)}")
        print(f"  Keys are similar: {sum(missed_analysis['key_similar'])} / {len(missed_analysis)}")
        print(f"  Keys don't match: {sum(~missed_analysis['key_match'])} / {len(missed_analysis)}")
        
        print(f"\nSeason Year Analysis:")
        print(f"  Season year matches: {sum(missed_analysis['season_year_match'])} / {len(missed_analysis)}")
        season_diffs = missed_analysis['season_year_diff'].dropna()
        if len(season_diffs) > 0:
            print(f"  Mean season diff: {season_diffs.mean():.2f}")
            print(f"  Max season diff: {season_diffs.max():.0f}")
            print(f"  Distribution: {season_diffs.value_counts().sort_index().to_dict()}")
        
        print(f"\nBirth Year Analysis:")
        birth_diffs = missed_analysis['birth_year_diff'].dropna()
        if len(birth_diffs) > 0:
            print(f"  Mean birth diff: {birth_diffs.mean():.2f}")
            print(f"  Max birth diff: {birth_diffs.max():.0f}")
        
        # Root cause analysis
        print("\n" + "=" * 80)
        print("ROOT CAUSE ANALYSIS")
        print("=" * 80)
        
        # Case 1: Keys match but still missed (window size issue)
        keys_match_but_missed = sum(missed_analysis['key_match'])
        print(f"\nCase 1: Keys match but still missed (likely window size issue): {keys_match_but_missed} / {len(missed_analysis)}")
        if keys_match_but_missed > 0:
            print("  → These matches have the same sorting key but are not within window distance")
            print("  → Solution: Increase window size or use composite key")
        
        # Case 2: Keys don't match but are similar (normalization issue)
        keys_similar_but_not_match = sum(
            missed_analysis['key_similar'] & ~missed_analysis['key_match']
        )
        print(f"\nCase 2: Keys are similar but don't match (normalization issue): {keys_similar_but_not_match} / {len(missed_analysis)}")
        if keys_similar_but_not_match > 0:
            print("  → These matches have similar keys but slight differences prevent exact match")
            print("  → Solution: Improve name normalization or use fuzzy matching")
        
        # Case 3: Keys are completely different (fundamental mismatch)
        keys_different = sum(
            ~missed_analysis['key_match'] & ~missed_analysis['key_similar']
        )
        print(f"\nCase 3: Keys are completely different (fundamental mismatch): {keys_different} / {len(missed_analysis)}")
        if keys_different > 0:
            print("  → These matches have very different name prefixes")
            print("  → Solution: May need alternative blocking strategy (e.g., TokenBlocker, EmbeddingBlocker)")
        
        # Check if Enhanced StandardBlocker would catch these
        if 'candidates_enhanced' in globals():
            enhanced_std_set = set(zip(candidates_enhanced[edge_name]['id1'], 
                                      candidates_enhanced[edge_name]['id2']))
            caught_by_enhanced_std = sum(
                missed_analysis.apply(
                    lambda row: (row['id1'], row['id2']) in enhanced_std_set, axis=1
                )
            )
            print(f"\nWould Enhanced StandardBlocker catch these? {caught_by_enhanced_std} / {len(missed_analysis)}")
            if caught_by_enhanced_std > 0:
                print("  → Consider union with Enhanced StandardBlocker to recover these matches")
        
        # Save detailed analysis to CSV
        output_path = OUTPUT_DIR / 'blocking-evaluation' / 'missed_matches_analysis_Enhanced_SNB_LR.csv'
        missed_analysis.to_csv(output_path, index=False)
        print(f"\nDetailed analysis saved to: {output_path}")
        
    else:
        print("No missed matches found! All true matches are captured.")


=== Analyzing Missed Matches for Enhanced SortedNeighbourhoodBlocker (LR Edge) ===

Debug: Checking validation set structure...
  Total validation pairs: 101
  Label column unique values: [ True False]
  Label column value counts:
label
True     78
False    23
Name: count, dtype: int64

  After normalization - TRUE matches: 78

Debug: Candidate pairs info...
  Total candidate pairs: 1986017
  Candidate columns: ['id1', 'id2']
  Candidate set size: 1986017
  True matches found in candidates: 78 / 78

Total true matches in validation set: 78
True matches found in candidates: 78
True matches MISSED: 0

No missed matches found! All true matches are captured.


### 2.7 TokenBlocker — Name Tokens (Jaccard ≥ 0.2)

- Uses whitespace token overlap on `full_name` to capture order and suffix variations.
- Provides a higher-recall channel that can be unioned with the previous blockers.


In [92]:
# 2.7 Materialize and evaluate TokenBlocker (name tokens)
from PyDI.entitymatching.blocking import TokenBlocker

token_blockers_name = {}
candidates_token = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: TokenBlocker (full_name tokens, min token len ≥ 2) ===")
    blocker = TokenBlocker(
        df_left=left_df,
        df_right=right_df,
        column='full_name',
        id_column='_rid',
        min_token_len=2,
        output_dir=str(OUTPUT_DIR / 'blocking-evaluation')
        # tokenizer=whitespace_tokenizer  # optional: supply your own function
    )
    token_blockers_name[edge_name] = blocker

    cand_df = blocker.materialize()
    candidates_token[edge_name] = cand_df

    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(cand_df) / total_pairs) if total_pairs else float('nan')
    print(f"  Generated: {len(cand_df):,} candidates | reduction ratio={rr:.6f}")

# Evaluation using common function defined in section 2.2.2
blocking_metrics_token = evaluate_blocking_strategy(
    candidates_dict=candidates_token,
    blockers_dict=token_blockers_name,
    edges_dict=edges,
    strategy_name="TokenBlocker",
    output_dir=OUTPUT_DIR / 'blocking-evaluation'
)

[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Creating token index for dataset1: 106553 records



=== LR: TokenBlocker (full_name tokens, min token len ≥ 2) ===


[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Creating token index for dataset2: 15215 records
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 12573 token keys for first dataset
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 3709 token keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Joining token keys: 12573 x 3709 tokens
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 3508 blocks from token keys
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Token frequency distribution:
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 242         4
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 196         1
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 140         9
[DEBUG] PyDI.entitymatching.bloc

  Generated: 5,607,705 candidates | reduction ratio=0.996541

=== LS: TokenBlocker (full_name tokens, min token len ≥ 2) ===


[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Creating token index for dataset2: 6743 records
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 12573 token keys for first dataset
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 2093 token keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Joining token keys: 12573 x 2093 tokens
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 2086 blocks from token keys
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Token frequency distribution:
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 103         1
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 102         4
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 81          9
[DEBUG] PyDI.entitymatching.block

  Generated: 2,277,960 candidates | reduction ratio=0.996830

=== LR: validation evaluation (TokenBlocker) ===


[INFO ] root -   Pair Completeness: 0.974
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.996541
[INFO ] root -   True Matches Found: 76/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 0.9743589743589743
  pair_quality: 1.3552781396310968e-05
  reduction_ratio: 0.9965410242244699
  total_candidates: 5607705
  total_possible_pairs: 1621203895
  true_positives_found: 76
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T18:02:45.143913
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (TokenBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.996830
[INFO ] root -   True Matches Found: 58/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 2.5461377723928428e-05
  reduction_ratio: 0.9968295036881251
  total_candidates: 2277960
  total_possible_pairs: 718486879
  true_positives_found: 58
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:04:38.439750
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


### TokenBlocker Evaluation Summary

- **Coverage**  
  - `LR`: Pair Completeness 0.974 (76/78 matches found).  
  - `LS`: Pair Completeness 1.000 (58/58 matches found).  
  - Token-based blocking is nearly lossless on validation GT, a large recall gain over initials-only keys.

- **Candidate Volume & Reduction Ratio**  
  - `LR`: 5,607,705 candidates, reduction ratio ≈ 0.9965 (removes only 0.35% of full Cartesian product).  
  - `LS`: 2,277,960 candidates, reduction ratio ≈ 0.9968.  
  - Millions of pairs still flow downstream; runtime remains heavy.

- **Pair Quality**  
  - Pair Quality ~1e-5 for both edges, reflecting very sparse true matches among candidates.  
  - High-precision scoring or additional filters are required before manual labeling or rule-based matching.

- **Implications & Actions**  
  - Token blocker should be combined with stricter blockers (e.g., name initials) to balance recall and efficiency.  
  - Consider tightening parameters (`min_token_len`, dropping common tokens, `max_token_freq`) to trade a little recall for large runtime savings.  
  - Downstream similarity scoring or classification is mandatory to raise precision before annotation.

### 2.8 Enhanced TokenBlocker (Dual Tokenization Strategy)

This section implements an improved version of TokenBlocker using a **dual tokenization strategy**:

- **Strategy**: Run TokenBlocker twice and union the results
  1. TokenBlocker on original `full_name` column (preserves original tokenization)
  2. TokenBlocker on normalized `full_name_normalized` column (handles encoding issues)
  3. Union both candidate sets to maximize recall

- **Rationale**: Some matches are detectable only with original names, while others require normalized names. The union ensures all matches are captured.

- **Results**: 
  - Achieves 100% recall for both LR and LS edges
  - LR: 5,974,146 candidates (union of 5.6M original + 5.7M normalized)
  - LS: 2,277,972 candidates (union of 2.3M original + 2.2M normalized)


#### 2.8a Materialize Enhanced TokenBlocker (with name normalization)


In [100]:
# 2.9a Materialize Enhanced TokenBlocker (dual tokenization: original + normalized)
# Strategy: Union candidates from both original full_name and normalized full_name_normalized
# This maximizes recall by capturing matches from both token sets
from PyDI.entitymatching.blocking import TokenBlocker
import pandas as pd

token_blockers_enhanced = {}
candidates_token_enhanced = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: Enhanced TokenBlocker (dual tokenization) ===")
    
    # Verify full_name_normalized column exists (created in section 2.2)
    for label, df_obj in [("left", left_df), ("right", right_df)]:
        if "full_name_normalized" not in df_obj.columns:
            raise ValueError(f"full_name_normalized column not found for {label} table. Please run section 2.2 (Name Normalization) first.")
    
    # Step 1: TokenBlocker on original full_name (preserves original tokenization)
    blocker_original = TokenBlocker(
        df_left=left_df,
        df_right=right_df,
        column='full_name',
        id_column='_rid',
        min_token_len=2,
        output_dir=str(OUTPUT_DIR / 'blocking-evaluation')
    )
    cand_original = blocker_original.materialize()
    
    # Step 2: TokenBlocker on normalized full_name_normalized (handles encoding issues)
    blocker_normalized = TokenBlocker(
        df_left=left_df,
        df_right=right_df,
        column='full_name_normalized',
        id_column='_rid',
        min_token_len=2,
        output_dir=str(OUTPUT_DIR / 'blocking-evaluation')
    )
    cand_normalized = blocker_normalized.materialize()
    
    # Step 3: Union both candidate sets (deduplicate by id1, id2)
    all_candidates = pd.concat([cand_original, cand_normalized], ignore_index=True)
    candidates_union = all_candidates.drop_duplicates(subset=['id1', 'id2']).reset_index(drop=True)
    
    token_blockers_enhanced[edge_name] = blocker_original  # Use original blocker for evaluation reference
    candidates_token_enhanced[edge_name] = candidates_union

    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(candidates_union) / total_pairs) if total_pairs else float('nan')
    print(f"  Original: {len(cand_original):,} | Normalized: {len(cand_normalized):,} | Union: {len(candidates_union):,} | RR: {rr:.6f}")


[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Creating token index for dataset1: 106553 records



=== LR: Enhanced TokenBlocker (dual tokenization) ===


[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Creating token index for dataset2: 15215 records
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 12573 token keys for first dataset
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 3709 token keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Joining token keys: 12573 x 3709 tokens
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 3508 blocks from token keys
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Token frequency distribution:
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 242         4
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 196         1
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 140         9
[DEBUG] PyDI.entitymatching.bloc

  Original: 5,607,705 | Normalized: 5,733,797 | Union: 5,974,146 | RR: 0.996315

=== LS: Enhanced TokenBlocker (dual tokenization) ===


[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Creating token index for dataset2: 6743 records
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 12573 token keys for first dataset
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 2093 token keys for second dataset
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Joining token keys: 12573 x 2093 tokens
[INFO ] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - created 2086 blocks from token keys
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Token frequency distribution:
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - Size Frequency
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 103         1
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 102         4
[DEBUG] PyDI.entitymatching.blocking.token_blocking.TokenBlocker - 81          9
[DEBUG] PyDI.entitymatching.block

  Original: 2,277,960 | Normalized: 2,197,973 | Union: 2,277,972 | RR: 0.996829


#### 2.8b Validate Enhanced TokenBlocker with ground-truth splits


In [101]:
# 2.9b Validate Enhanced TokenBlocker with Ground-Truth Splits
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_token_enhanced = evaluate_blocking_strategy(
    candidates_dict=candidates_token_enhanced,
    blockers_dict=token_blockers_enhanced,
    edges_dict=edges,
    strategy_name="Enhanced TokenBlocker",
    output_dir=OUTPUT_DIR / 'blocking-evaluation'
)



=== LR: validation evaluation (Enhanced TokenBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.996315
[INFO ] root -   True Matches Found: 78/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 1.3056259421848746e-05
  reduction_ratio: 0.9963149940495301
  total_candidates: 5974146
  total_possible_pairs: 1621203895
  true_positives_found: 78
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T18:56:34.975990
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (Enhanced TokenBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.996829
[INFO ] root -   True Matches Found: 58/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 2.5461243597375208e-05
  reduction_ratio: 0.9968294869863588
  total_candidates: 2277972
  total_possible_pairs: 718486879
  true_positives_found: 58
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:58:21.374639
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


### 2.9 EmbeddingBlocker — Sentence similarity on `full_name`

Uses sentence-transformer embeddings to retrieve the top-k nearest neighbours per record. Provides high recall on challenging variants but is slower and requires downloading the embedding model.

#### 2.9a Materialize EmbeddingBlocker (MiniLM, cosine, top_k=20)

In [95]:
from PyDI.entitymatching.blocking import EmbeddingBlocker

embedding_blockers = {}
candidates_embedding = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: EmbeddingBlocker (full_name, top_k=20) ===")

    blocker = EmbeddingBlocker(
        df_left=left_df,
        df_right=right_df,
        text_cols=["full_name"],                 # build embeddings from player names
        model="sentence-transformers/all-MiniLM-L6-v2",
        index_backend="sklearn",                # faiss/hnsw backends available if installed
        metric="cosine",
        top_k=20,
        id_column="_rid",
        output_dir=str(OUTPUT_DIR / "blocking-evaluation")
    )
    embedding_blockers[edge_name] = blocker

    cand_df = blocker.materialize()
    candidates_embedding[edge_name] = cand_df

    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(cand_df) / total_pairs) if total_pairs else float("nan")
    print(f"  Generated: {len(cand_df):,} candidates | reduction ratio={rr:.6f}")


=== LR: EmbeddingBlocker (full_name, top_k=20) ===


[INFO ] PyDI.entitymatching.blocking.embedding.EmbeddingBlocker - Initialized EmbeddingBlocker with sklearn backend, top_k=20, threshold=0.3
[DEBUG] PyDI.entitymatching.blocking.embedding.EmbeddingBlocker - Computing embeddings for datasets...
[DEBUG] PyDI.entitymatching.blocking.embedding.EmbeddingBlocker - Creating embeddings for dataset1: 106553 records
[INFO ] sentence_transformers.SentenceTransformer - Use pytorch device_name: mps
[INFO ] sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
[DEBUG] urllib3.connectionpool - Resetting dropped connection: huggingface.co
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json HTTP/1.1" 200 0
[DEBU

  Generated: 2,130,317 candidates | reduction ratio=0.998686

=== LS: EmbeddingBlocker (full_name, top_k=20) ===


[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json HTTP/1.1" 200 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json HTTP/1.1" 200 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models

  Generated: 2,128,676 candidates | reduction ratio=0.997037


#### 2.9b Validate EmbeddingBlocker with ground-truth splits

In [96]:
# 2.10b Validate EmbeddingBlocker with Ground-Truth Splits
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_embedding = evaluate_blocking_strategy(
    candidates_dict=candidates_embedding,
    blockers_dict=embedding_blockers,
    edges_dict=edges,
    strategy_name="EmbeddingBlocker",
    output_dir=OUTPUT_DIR / "blocking-evaluation"
)


=== LR: validation evaluation (EmbeddingBlocker) ===


[INFO ] root -   Pair Completeness: 0.679
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.998686
[INFO ] root -   True Matches Found: 53/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 0.6794871794871795
  pair_quality: 2.4878926469628698e-05
  reduction_ratio: 0.998685966024033
  total_candidates: 2130317
  total_possible_pairs: 1621203895
  true_positives_found: 53
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T18:15:45.167696
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (EmbeddingBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.997037
[INFO ] root -   True Matches Found: 58/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 2.724698357100846e-05
  reduction_ratio: 0.9970372792291451
  total_candidates: 2128676
  total_possible_pairs: 718486879
  true_positives_found: 58
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:16:40.936413
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


#### 2.9c EmbeddingBlocker Evaluation Summary

- **Coverage**
  - `LR`: Pair Completeness 0.679 (53/78 matches). Recall is substantially lower than SNB and TokenBlocker; many LR matches remain uncovered.
  - `LS`: Pair Completeness 1.000 (58/58). The model captures all validation matches for LS.

- **Candidate Volume**
  - `LR`: 2,130,317 candidates, reduction ratio ≈ 0.99869 (≈0.31% of the full Cartesian). Despite high recall, the channel still produces millions of pairs; runtime remains heavy.
  - `LS`: 2,128,676 candidates, reduction ratio ≈ 0.99704. Similar scale to LR.

- **Pair Quality**
  - `LR`: Pair Quality 2.49e-05; `LS`: 2.72e-05. True matches remain very sparse in the candidate pool, so scoring/ranking is still essential.

- **Takeaways**
  - Embedding-based blocking provides excellent recall for LS but struggles on LR—likely due to noisier naming conventions or longer historical coverage on the Lahman side.
  - Candidate volume is comparable to token blocking but with lower LR recall; tuning is required (`top_k`, similarity threshold, or model choice) to get the desired trade-off.
  - Consider switching to a faster index backend (`faiss`) and experimenting with higher `top_k`, relaxed thresholds, or multi-column embeddings (e.g., name + birth year) to boost LR coverage without exploding candidates.

#### 2.9e Enhanced EmbeddingBlocker (MPNet, top_k=80)

Improved version using normalized names (`full_name_normalized`) and higher `top_k=80` to improve recall on the challenging LR edge. Uses the fastest model (`all-MiniLM-L6-v2`) for computational efficiency.


In [97]:
# 2.9e Materialize Enhanced EmbeddingBlocker (reusing TokenBlocker normalization, fast model)
from PyDI.entitymatching.blocking import EmbeddingBlocker

embedding_blockers_enhanced = {}
candidates_embedding_enhanced = {}

for edge_name, (left_df, right_df, _, _) in edges.items():
    print(f"\n=== {edge_name}: Enhanced EmbeddingBlocker (reusing TokenBlocker normalization, fast model) ===")
    
    # Check if full_name_normalized column exists (created by Enhanced TokenBlocker)
    # If not, we need to create it (reuse TokenBlocker's normalization logic)
    if "full_name_normalized" not in left_df.columns or "full_name_normalized" not in right_df.columns:
        print(f"  Warning: full_name_normalized column not found. Please run Enhanced TokenBlocker (2.8a) first.")
        print(f"  Skipping {edge_name} edge.")
        continue
    
    print(f"  Using full_name_normalized column (created by Enhanced TokenBlocker)")
    
    # Use fastest model: all-MiniLM-L6-v2 (smaller and faster than MPNet)
    # Reuse the normalized name column from TokenBlocker
    blocker = EmbeddingBlocker(
        df_left=left_df,
        df_right=right_df,
        text_cols=["full_name_normalized"],      # Reuse normalized column from TokenBlocker
        model="sentence-transformers/all-MiniLM-L6-v2",  # Fastest model (smaller than MPNet)
        index_backend="sklearn",                 # faiss/hnsw backends available if installed
        metric="cosine",
        top_k=80,                                 # Keep high top_k to improve recall
        threshold=0.2,                            # Lower threshold to catch more matches
        id_column="_rid",
        output_dir=str(OUTPUT_DIR / "blocking-evaluation")
    )
    embedding_blockers_enhanced[edge_name] = blocker

    cand_df = blocker.materialize()
    candidates_embedding_enhanced[edge_name] = cand_df

    total_pairs = len(left_df) * len(right_df)
    rr = 1.0 - (len(cand_df) / total_pairs) if total_pairs else float("nan")
    print(f"  Generated: {len(cand_df):,} candidates | reduction ratio={rr:.6f}")



=== LR: Enhanced EmbeddingBlocker (reusing TokenBlocker normalization, fast model) ===
  Using full_name_normalized column (created by Enhanced TokenBlocker)


[INFO ] PyDI.entitymatching.blocking.embedding.EmbeddingBlocker - Initialized EmbeddingBlocker with sklearn backend, top_k=80, threshold=0.2
[DEBUG] PyDI.entitymatching.blocking.embedding.EmbeddingBlocker - Computing embeddings for datasets...
[DEBUG] PyDI.entitymatching.blocking.embedding.EmbeddingBlocker - Creating embeddings for dataset1: 106553 records
[INFO ] sentence_transformers.SentenceTransformer - Use pytorch device_name: mps
[INFO ] sentence_transformers.SentenceTransformer - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
[DEBUG] urllib3.connectionpool - Resetting dropped connection: huggingface.co
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json HTTP/1.1" 200 0
[DEBU

  Generated: 8,524,240 candidates | reduction ratio=0.994742

=== LS: Enhanced EmbeddingBlocker (reusing TokenBlocker normalization, fast model) ===
  Using full_name_normalized column (created by Enhanced TokenBlocker)


[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json HTTP/1.1" 200 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config_sentence_transformers.json HTTP/1.1" 200 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json HTTP/1.1" 307 0
[DEBUG] urllib3.connectionpool - https://huggingface.co:443 "HEAD /api/resolve-cache/models

  Generated: 8,524,188 candidates | reduction ratio=0.988136


In [98]:
# 2.10d Validate Enhanced EmbeddingBlocker
# Uses common evaluation function defined in section 2.2.2
blocking_metrics_embedding_enhanced = evaluate_blocking_strategy(
    candidates_dict=candidates_embedding_enhanced,
    blockers_dict=embedding_blockers_enhanced,
    edges_dict=edges,
    strategy_name="Enhanced EmbeddingBlocker",
    output_dir=OUTPUT_DIR / "blocking-evaluation"
)



=== LR: validation evaluation (Enhanced EmbeddingBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.994742
[INFO ] root -   True Matches Found: 78/78
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 9.150375869285707e-06
  reduction_ratio: 0.9947420308905686
  total_candidates: 8524240
  total_possible_pairs: 1621203895
  true_positives_found: 78
  total_true_pairs: 78
  evaluation_timestamp: 2025-11-16T18:36:43.619245
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']

=== LS: validation evaluation (Enhanced EmbeddingBlocker) ===


[INFO ] root -   Pair Completeness: 1.000
[INFO ] root -   Pair Quality:      0.000
[INFO ] root -   Reduction Ratio:   0.988136
[INFO ] root -   True Matches Found: 58/58
[INFO ] root - Blocking evaluation complete!


  pair_completeness: 1.0
  pair_quality: 6.804167153516558e-06
  reduction_ratio: 0.9881359169538849
  total_candidates: 8524188
  total_possible_pairs: 718486879
  true_positives_found: 58
  total_true_pairs: 58
  evaluation_timestamp: 2025-11-16T18:40:30.363883
  output_files: ['/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_evaluation_summary.json', '/Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/blocking-evaluation/blocking_detailed_results.csv']


## 3. Blocking Strategy Comparison and Final Selection

### 3.1 Performance Comparison Tables

#### LR Edge (Lahman ↔ Reference) Performance

| Blocking Strategy | Pair Completeness (Recall) | Candidate Pairs | Reduction Ratio | Pair Quality |
|------------------|---------------------------|-----------------|----------------|--------------|
| StandardBlocker (Baseline) | 70.5% (55/78) | 449,131 | 99.97% | 1.22e-4 |
| Enhanced StandardBlocker | 100% (78/78) | 18,790,156 | 98.84% | 4.15e-6 |
| SortedNeighbourhoodBlocker (Baseline) | 84.6% (66/78) | 434,493 | 99.97% | 1.52e-4 |
| Enhanced SortedNeighbourhoodBlocker | 100% (78/78) | 1,986,017 | 99.88% | 3.93e-5 |
| TokenBlocker (Baseline) | 97.4% (76/78) | 5,607,705 | 99.65% | 1.36e-5 |
| Enhanced TokenBlocker | 100% (78/78) | 5,974,146 | 99.63% | 1.31e-5 |
| EmbeddingBlocker (Baseline) | 67.9% (53/78) | 2,130,317 | 99.87% | 2.49e-5 |
| Enhanced EmbeddingBlocker | 100% (78/78) | 8,524,240 | 99.47% | 9.15e-6 |

#### LS Edge (Lahman ↔ Savant) Performance

| Blocking Strategy | Pair Completeness (Recall) | Candidate Pairs | Reduction Ratio | Pair Quality |
|------------------|---------------------------|-----------------|----------------|--------------|
| StandardBlocker (Baseline) | 46.6% (27/58) | 193,269 | 99.97% | 1.40e-4 |
| Enhanced StandardBlocker | 100% (58/58) | 215,708 | 99.97% | 2.69e-4 |
| SortedNeighbourhoodBlocker (Baseline) | 94.8% (55/58) | 219,574 | 99.97% | 2.50e-4 |
| Enhanced SortedNeighbourhoodBlocker | 100% (58/58) | 964,403 | 99.87% | 6.01e-5 |
| TokenBlocker (Baseline) | 100% (58/58) | 2,277,960 | 99.68% | 2.55e-5 |
| Enhanced TokenBlocker | 100% (58/58) | 2,277,972 | 99.68% | 2.55e-5 |
| EmbeddingBlocker (Baseline) | 100% (58/58) | 2,130,317 | 99.70% | 2.49e-5 |
| Enhanced EmbeddingBlocker | 100% (58/58) | 8,524,188 | 98.81% | 1.17e-5 |

### 3.2 Key Findings

1. **LR Edge is More Challenging**: Most blocking strategies achieve lower recall on LR compared to LS, indicating that matching between Lahman and Reference datasets is more difficult due to name variations, encoding issues, and data quality differences.

2. **LS Edge is Easier**: Multiple strategies achieve 100% recall on LS, suggesting that matching between Lahman and Savant datasets is more straightforward.

3. **Candidate Volume Varies Dramatically**: 
   - Smallest for LR: Enhanced TokenBlocker (5.9M candidates)
   - Smallest for LS: Enhanced StandardBlocker (215K candidates)
   - Range: 85x difference between smallest and largest strategies

4. **Name Normalization is Critical**: Enhanced versions consistently outperform baseline versions, demonstrating the importance of unified name normalization (handling UTF-8 encoding, Unicode normalization, backslash escapes, and suffix removal).

5. **Trade-off Between Recall and Efficiency**:
   - High Recall (100%): Enhanced StandardBlocker, Enhanced SortedNeighbourhoodBlocker, Enhanced TokenBlocker, Enhanced EmbeddingBlocker
   - Moderate Recall (92-97%): TokenBlocker (baseline)
   - Low Recall (<85%): StandardBlocker (baseline), EmbeddingBlocker (baseline)

### 3.3 Final Strategy: Hybrid Approach

**Selected Strategy: Edge-Specific Blocking**

We adopt a **hybrid blocking strategy** that uses different blocking methods for different edges to optimize both recall and computational efficiency:

- **LR Edge**: Enhanced TokenBlocker
  - Recall: 100% (78/78 matches)
  - Candidate pairs: 5,974,146
  - Rationale: Achieves perfect recall with 68% fewer candidates than Enhanced StandardBlocker (18.8M candidates)

- **LS Edge**: Enhanced StandardBlocker
  - Recall: 100% (58/58 matches)
  - Candidate pairs: 215,708
  - Rationale: Achieves perfect recall with 91% fewer candidates than Enhanced TokenBlocker (2.3M candidates)

**Why Hybrid Strategy?**

1. **Optimal Performance per Edge**: Each edge has different characteristics. LR edge benefits from token-based blocking which handles name variations better, while LS edge benefits from standard blocking which is more efficient for this dataset.

2. **Computational Efficiency**: The hybrid approach reduces total candidate pairs from 19.0M (uniform Enhanced StandardBlocker) to 6.2M, achieving a 67% reduction while maintaining 100% recall on both edges.

3. **Maintains Perfect Recall**: Both selected strategies achieve 100% recall on their respective edges, ensuring no true matches are lost during blocking.

4. **Balanced Resource Usage**: 
   - LR edge: 5.9M candidates (manageable for token-based matching)
   - LS edge: 215K candidates (very efficient for standard matching)

**Total Impact**:
- Total candidate pairs: 6,189,854 (vs 19,005,864 with uniform Enhanced StandardBlocker strategy)
- Total savings: 12,816,010 candidate pairs (67% reduction)
- Recall: 100% on both edges
- Computational cost: Significantly reduced while maintaining perfect coverage

### 3.5 Hybrid Blocking Strategy: Select and Export Candidate Pairs

In [99]:
# LR Edge: Enhanced TokenBlocker (100% recall, 5.9M candidates)
# LS Edge: Enhanced StandardBlocker (100% recall, 215K candidates)

import pandas as pd

# Select optimal blocking strategy for each edge
candidates_for_matching = {}
blockers_for_matching = {}

# LR Edge: Enhanced TokenBlocker
if 'candidates_token_enhanced' in globals() and 'LR' in candidates_token_enhanced:
    candidates_for_matching['LR'] = candidates_token_enhanced['LR']
    blockers_for_matching['LR'] = token_blockers_enhanced['LR']
else:
    raise ValueError("Enhanced TokenBlocker not found. Please run section 2.8a first.")

# LS Edge: Enhanced StandardBlocker
if 'candidates_enhanced' in globals() and 'LS' in candidates_enhanced:
    candidates_for_matching['LS'] = candidates_enhanced['LS']
    blockers_for_matching['LS'] = standard_blockers_enhanced['LS']
else:
    raise ValueError("Enhanced StandardBlocker not found. Please run section 2.4 first.")

# Save candidate pairs to CSV files for next steps
print("Hybrid Blocking Strategy - Candidate Pairs:")
for edge_name in ['LR', 'LS']:
    cand_df = candidates_for_matching[edge_name]
    output_file = OUTPUT_DIR / f'candidates_hybrid_{edge_name}.csv'
    cand_df.to_csv(output_file, index=False)
    print(f"  {edge_name}: {len(cand_df):,} candidates -> {output_file}")

total_candidates = sum(len(cand) for cand in candidates_for_matching.values())
print(f"\nTotal candidate pairs: {total_candidates:,}")
print("Ready for matching phase. Use 'candidates_for_matching' dictionary.")


Hybrid Blocking Strategy - Candidate Pairs:
  LR: 5,733,797 candidates -> /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/candidates_hybrid_LR.csv
  LS: 215,708 candidates -> /Users/zhangzihan/Desktop/WBI_project/Schema_Mapped_Datasets/data/output/workflow/candidates_hybrid_LS.csv

Total candidate pairs: 5,949,505
Ready for matching phase. Use 'candidates_for_matching' dictionary.


### 3.4 Next Steps

With the hybrid blocking strategy selected, proceed to the **matching phase**:

1. **Implement Rule-Based Matcher**: Use `RuleBasedMatcher` with similarity comparators (Levenshtein, Jaccard, DateComparator)
2. **Tune Parameters**: Adjust weights and threshold on validation set
3. **Evaluate Matching Quality**: Assess precision, recall, and F1-score
4. **Apply Global Matching**: Enforce one-to-one constraints using `GreedyOneToOneMatchingAlgorithm`
5. **Final Evaluation**: Test on held-out test set

**Note**: The blocking phase ensures that all true matches are included in the candidate pairs. The matching phase will then determine which candidate pairs are actual matches based on similarity scores.


In [ ]:
import pandas as pd
import unicodedata
from collections import defaultdict

MANUAL_CASES_DIR = OUTPUT_DIR / 'gt' / 'manual_cases'
MANUAL_CASES_DIR.mkdir(parents=True, exist_ok=True)

MANUAL_ERROR_CASES = defaultdict(set)
for edge in ['LR', 'LS']:
    manual_path = MANUAL_CASES_DIR / f'manual_cases_{edge}.csv'
    if manual_path.exists():
        manual_df = pd.read_csv(manual_path)
        if {'id1', 'id2'}.issubset(manual_df.columns):
            MANUAL_ERROR_CASES[edge] = set(zip(manual_df['id1'], manual_df['id2']))


def _strip_accents(text: str) -> str:
    if not isinstance(text, str):
        return ''
    normalized = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in normalized if unicodedata.category(ch) != 'Mn')


def _has_accent_or_suffix_variant(row: pd.Series, left_tag: str, right_tag: str) -> bool:
    left = str(row.get(f'full_name_{left_tag}', '')).lower().strip()
    right = str(row.get(f'full_name_{right_tag}', '')).lower().strip()
    if not left or not right:
        return False
    accent_left = _strip_accents(left)
    accent_right = _strip_accents(right)
    if accent_left and accent_left == accent_right and left != right:
        return True
    suffix_tokens = {'jr', 'sr', 'ii', 'iii', 'iv', 'v'}
    left_suffix = any(left.endswith(f' {s}') for s in suffix_tokens)
    right_suffix = any(right.endswith(f' {s}') for s in suffix_tokens)
    if left_suffix != right_suffix:
        def _strip_suffix(text: str) -> str:
            parts = [tok for tok in text.split() if tok not in suffix_tokens]
            return ' '.join(parts)
        base_left = _strip_accents(_strip_suffix(left))
        base_right = _strip_accents(_strip_suffix(right))
        return base_left == base_right
    return False


def _has_birth_conflict(row: pd.Series, left_tag: str, right_tag: str, threshold: int = 2) -> bool:
    by_l = pd.to_numeric(row.get(f'birth_year_{left_tag}'), errors='coerce')
    by_r = pd.to_numeric(row.get(f'birth_year_{right_tag}'), errors='coerce')
    if pd.isna(by_l) or pd.isna(by_r):
        return False
    return abs(by_l - by_r) >= threshold

